<a href="https://colab.research.google.com/github/crim-ca/geoimagenet/blob/master/model_packaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook explains how to package your model using the thelper library.

In [1]:
%%bash
pip3 --quiet install  torch torchvision pillow gitpython lz4 matplotlib numpy pyyaml scikit-learn six tqdm h5py opencv-python googledrivedownloader pretrainedmodels albumentations pyyaml
pip3 --quiet install affine geojson shapely pyproj hdf5plugin
pip uninstall thelper
rm -rf ./thelper
git clone https://github.com/sfoucher/thelper
pip3 install --quiet -e ./thelper

Cloning into 'thelper'...


The first time you are installing thelper, you'll need to restart the current session

In [ ]:
import os
os.kill(os.getpid(), 9)

This function  

In [2]:
from google_drive_downloader import GoogleDriveDownloader as gdd
import os.path as osp
import sys


def maybe_download_and_extract(file_id, dest_path):
    filename = dest_path.split('/')[-1]
    file_path = dest_path
    download_dir= osp.dirname(osp.abspath(dest_path))
    if not os.path.isfile(dest_path):
      gdd.download_file_from_google_drive(file_id= file_id, dest_path= file_path)
      print("Download finished. Extracting files.")

      if file_path.endswith(".zip"):
          # Unpack the zip-file.
          zipfile.ZipFile(file=file_path, mode="r").extractall(download_dir)
      elif file_path.endswith((".tar.gz", ".tgz")):
          # Unpack the tar-ball.
          tarfile.open(name=file_path, mode="r:gz").extractall(download_dir)
      print("Done.")
    else:
        print("Data has apparently already been downloaded and unpacked.")

In [1]:
import os
import torch
import torchvision
import thelper
# initialisation des logs (pour TOUT imprimer ce qui se passe)
thelper.utils.init_logger()

In [ ]:
class_names = ["AgriculturalLand", "BarrenLand", "ForestLand", "RangeLand", "UrbanLand", "Water"]
save_dir = 'sessions'
task_config = {
    "type": "thelper.tasks.Classification",
    "params": {
        "class_names": class_names,
        "input_key": "data",
        "label_key": "label"
    }
}

Define the data loader:
*   List item
*   List item



In [ ]:
datasets_config = {
    # loader for the test data
    "deepglobe_test": {
        "type": "thelper.data.geo.ImageFolderGDataset",
        "params": {"root": "/home/sfoucher/DEV/geoimagenet/dataset_test/deepglobe_classif",
                    "image_key": "image",
                    "channels": [1,2,3]

                    },
        "task": task_config
    }
}

In [ ]:
loaders_config = {
    "batch_size": 1,
    # ci-dessous, la liste des transformations à appliquer à toutes les images...
    "base_transforms": [

        {
            "operation": "thelper.transforms.Resize",
            "params": {"dsize": [224, 224]},
        },
        {
            "operation": "thelper.transforms.NormalizeMinMax",
            "params": {
                "min": [0, 0, 0],
                "max": [255, 255, 255]
            },
        },
        {
            "operation": "thelper.transforms.NormalizeZeroMeanUnitVar",
            "params": {
                "mean": [0.485, 0.456, 0.406],
                "std": [0.229, 0.224, 0.225]
            },
        },
        {
            "operation": "torchvision.transforms.ToTensor",
        },
    ],

    "test_split": {
        "deepglobe_test": 1.0
    },
}

In [ ]:
model_config = {
    "type": torchvision.models.resnet18,
    "params": {"pretrained": True},
    "state_dict": '/home/sfoucher/DEV/pytorch-segmentation/model/resnet_2020-07-16 14:36:58.634014/model.pth',
    "task": task_config

}

In [ ]:
config = {"name": 'deepglobe-resnet-18', "model": model_config, "datasets": datasets_config, "loaders": loaders_config,
              "trainer": trainer_config}

In [ ]:

export_config = {
    "ckpt_name": "test-resnet18-imagenet.pth",
    "trace_name": "test-resnet18-imagenet.zip",
    "save_raw": True,
    "trace_input": "torch.rand(1, 3, 224, 224)",
    "task": {
        "type": "thelper.tasks.Classification",
        "params": {
            "class_names": "tests/meta/imagenet_classes.json",
            "input_key": "0",
            "label_key": "1"
        }
    }
}


# model = thelper.nn.create_model(config, task, save_dir=save_dir)
# model.eval()
thelper.cli.export_model(config, '/home/sfoucher/DEV/geoimagenet/pth')

class_mapping = [('AgriculturalLand', 223), ('BarrenLand', 252), ('ForestLand', 233), ('RangeLand', 229),
                  ('UrbanLand', 200), ('Water', 239)]





In [ ]:
update_model_class_mapping(class_mapping,
                            '/home/sfoucher/DEV/geoimagenet/pth/deepglobe-resnet-18/deepglobe-resnet-18.export.pth', None)
# success, model, buffer, exception = load_model('/home/sfoucher/DEV/geoimagenet/pth/deepglobe-unet/deepglobe-unet.export.pth')
# validate_model(model)